<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [ ]:
# Loading the dataset using sklearn
from sklearn.datasets import load_wine
data = load_wine()

# Displaying relevant information about the data
print(data['DESCR'][200:1420])

In [ ]:
# Separating data into feature variable X and target variable y respectively
from sklearn.model_selection import train_test_split
X = data['data']
y = data['target']

# Extracting the names of the features from data
features = data['feature_names']

# Splitting X & y into training and testing set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.90, random_state=50)

# Creating a dataframe of the data, for a visual check
df = pd.concat([pd.DataFrame(X), pd.DataFrame(y)], axis=1)
df.columns = np.concatenate((features, np.array(['label'])))
print("Shape of data =", df.shape)

# Printing the top 5 rows of the dataframe
df.head()

In [ ]:
# Instantiating the prediction model - an extra-trees regressor
from sklearn.ensemble import ExtraTreesRegressor
reg = ExtraTreesRegressor(random_state=50)

# Fitting the predictino model onto the training set
reg.fit(X_train, y_train)

# Checking the model's performance on the test set
print('R2 score for the model on test set =', reg.score(X_test, y_test))

In [ ]:

# Importing the module for LimeTabularExplainer
from lime import lime_tabular

# Instantiating the explainer object by passing in the training set,
# and the extracted features
explainer_lime = lime_tabular.LimeTabularExplainer(X_train,
                                                   feature_names=features,
                                                   verbose=True,
                                                   mode='regression')

In [ ]:

# Index corresponding to the test vector
i = 10

# Number denoting the top features
k = 5

# Calling the explain_instance method by passing in the:
#    1) ith test vector
#    2) prediction function used by our prediction model('reg' in this case)
#    3) the top features which we want to see, denoted by k

exp_lime = explainer_lime.explain_instance(
    X_test[i], reg.predict, num_features=k)

# Finally visualizing the explanations
exp_lime.show_in_notebook()

In [ ]:
import torch
from torchvision.datasets import ImageNet
import torchvision.models as models

model = models.resnet50(pretrained=True)

In [ ]:
import json
import urllib

# URL đến file chứa nhãn ImageNet
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"

# Tải nhãn từ URL
response = urllib.request.urlopen(LABELS_URL)
labels = json.loads(response.read().decode())

# In thử một số nhãn
print(labels[:10])  # Hiển thị 10 nhãn đầu tiên

In [ ]:
import skimage as ski
from skimage.transform import resize
from skimage.io import imread
from skimage import img_as_float

In [ ]:
image_reader_raw = imread(r'E:\2.Image\16.Image_net\Nam-doc01.jpg')
image_reader = img_as_float(image_reader_raw)
image_reader = resize(image_reader, (256, 256))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
image_normalized = (image_reader - mean) / std
# plt.imshow(image_reader)
tensor = torch.from_numpy(image_normalized).transpose(-1, 0).unsqueeze(0).float()
model.eval()
with torch.no_grad():
    predict = torch.softmax(model(tensor), -1)
    arg_label = torch.argmax(predict, -1)
    print(labels[arg_label], arg_label.item(), predict[0, arg_label].item())
    for x in predict.argsort()[0][-5:]:
        print(labels[x.item()], x.item(), predict[0, x].item())
plt.imshow(image_reader)

In [ ]:
def predict_fn(img):
    image_reader = img_as_float(img)
    image_reader = resize(image_reader, (256, 256))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image_normalized = (image_reader - mean) / std
    # plt.imshow(image_reader)
    tensor = torch.from_numpy(image_normalized).transpose(-1, 0).unsqueeze(0).float()
    model.eval()
    with torch.no_grad():
        predict = torch.softmax(model(tensor), -1)
        arg_label = torch.argmax(predict, -1)
        return predict.numpy()

print(predict_fn(image_reader_raw).argsort()[0][-5:])

In [ ]:
from lime import lime_image
import time
explainer = lime_image.LimeImageExplainer()
tmp = time.time()
print(image_reader_raw.shape)
image_reader = resize(image_reader_raw, (256, 256))
# Hide color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels
explanation = explainer.explain_instance(image_reader, predict_fn, top_labels=5, hide_color=0, num_samples=1000)
print (time.time() - tmp)